# Welcome to "ML4Recsys : Intro to content-based filtering" Notebook

In this notebook we will try recommend list of film based on one film that the user already watch, so the instruction is:

1. Read the data
2. Make the vector representation
3. Calculate the similarity betweenfilm based on the vector representation

## Read the data

In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1hJq19J44880FK4eKNv-msoOu366xX-Z2"}) #this is the file id
downloaded.GetContentFile('imdb_indonesian_movies_2.csv')
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df.head()

In [ ]:
# akan dilakukan content base filtering dari movie yagn tersedia berdasarkan ringkasan sinopsis
df['ringkasan_sinopsis']

0       Raden Mas Said putra sulung Tumenggung Wilarik...
1       Soe Hok Gie adalah seorang aktivis yang hidup ...
2       Guru Bangsa Tjokroaminoto menceritakan tentang...
3       POL menceritakan kisah hidup yang luar biasa d...
4       Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...
                              ...                        
1000    Winter in Tokyo berpusat pada kehidupan Ishida...
1001    Markonah melarikan diri ke Jakarta karena akan...
1002    Tempat aking lebih dari 36 jam, Last Night ada...
1003    Proyek baru ini adalah tentang seorang lelaki ...
1004    Atika (Meriam Bellina) mantan penyanyi tenar, ...
Name: ringkasan_sinopsis, Length: 1005, dtype: object

#Preprocessing

##Remove punctuation

In [ ]:
import string

for col in df.columns:
    for i in range(0,8):
        df[col][i] = df[col][i].translate(str.maketrans('', '', string.punctuation))
df

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...,...
1000,Winter in Tokyo,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Petualang Cinta,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,Last Night (III),"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Path of Light,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


##Convert to lowercase

In [ ]:
for col in df.columns:
    for i in range(df.shape[0]):
        df[col][i] = df[col][i].lower()
df

,judul_film,ringkasan_sinopsis,genre
0,sunan kalijaga,raden mas said putra sulung tumenggung wilarik...,drama
1,gie,soe hok gie adalah seorang aktivis yang hidup ...,drama
2,guru bangsa tjokroaminoto,guru bangsa tjokroaminoto menceritakan tentang...,drama
3,pol movie,pol menceritakan kisah hidup yang luar biasa d...,drama
4,sang pencerah,perjalanan pahlawan indonesia kh ahmad dahlan ...,drama
...,...,...,...
1000,winter in tokyo,winter in tokyo berpusat pada kehidupan ishida...,romantis
1001,petualang cinta,markonah melarikan diri ke jakarta karena akan...,romantis
1002,last night (iii),"tempat aking lebih dari 36 jam, last night ada...",romantis
1003,path of light,proyek baru ini adalah tentang seorang lelaki ...,romantis


##Stop Words Removal

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

listStopword =  set(stopwords.words('indonesian'))

for i in range(0,9):
    tokens = word_tokenize(df['ringkasan_sinopsis'][i])
    without_stopword = []
    for token in tokens:
        if token not in listStopword:
            without_stopword.append(token)
    df['ringkasan_sinopsis'][i] = ' '.join(without_stopword)
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,judul_film,ringkasan_sinopsis,genre
0,sunan kalijaga,raden mas said putra sulung tumenggung wilarik...,drama
1,gie,soe hok gie aktivis hidup enam puluhan terleta...,drama
2,guru bangsa tjokroaminoto,guru bangsa tjokroaminoto menceritakan lepas e...,drama
3,pol movie,pol menceritakan kisah hidup 3 pemain skateboa...,drama
4,sang pencerah,perjalanan pahlawan indonesia kh ahmad dahlan ...,drama
...,...,...,...
1000,winter in tokyo,winter in tokyo berpusat pada kehidupan ishida...,romantis
1001,petualang cinta,markonah melarikan diri ke jakarta karena akan...,romantis
1002,last night (iii),"tempat aking lebih dari 36 jam, last night ada...",romantis
1003,path of light,proyek baru ini adalah tentang seorang lelaki ...,romantis


##Test

In [ ]:
df['ringkasan_sinopsis'][0]

'raden mas said putra sulung tumenggung wilarikta kerajaan majapahit berkuasa wilayah tuban sekeluarga miskin menderita busung lapar prihatin hati nuraninya tergugah menolong mencoba diamdiam mengambil makanan lumbung orang tuanya perbuatan disetujui orang tuanya dihukum sekap gudang makanan kejadian rm said tumbuh dewasa betah tinggal rumah berkelana daerah daerah sanalah betapa penyelewengan kesewenangwenangan lurah munafik mengkambinghitamkan tumenggung menutupi kejahatannya laporan rm said ayahnya sadar dianggap sumber fitnah kelananya bertemu sunan bonang mencurahkan ilmunya rm said tapa pinggir kali berkat ketabahannya menghadapi cobaan rm said nur kekuatan ilahi diangkat wali terkenal deretan nama sembilan wali wali sanga nama sunan kalijaga'

## Vector Representation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# menghitung nilai tff idf dari setiap kata yang ada di plot
tfidf = TfidfVectorizer(use_idf=True) 
tfidf_matrix = tfidf.fit_transform(df['ringkasan_sinopsis'])  # membuat TF-IDF Matrix (no. of movies x every word in vocabulary)
tfidf_matrix.shape

(1005, 13518)

In [ ]:
print(tfidf_matrix)

  (0, 5167)	0.0993598131443519
  (0, 10989)	0.0993598131443519
  (0, 11329)	0.08424222453335829
  (0, 8668)	0.1155937750122814
  (0, 2477)	0.08675108258906822
  (0, 12561)	0.0577968875061407
  (0, 13259)	0.2694650300622691
  (0, 2565)	0.07590141875930392
  (0, 4586)	0.0993598131443519
  (0, 5570)	0.059209472533904496
  (0, 8881)	0.08424222453335829
  (0, 2208)	0.08028354023049417
  (0, 7820)	0.053263048707919716
  (0, 5871)	0.0993598131443519
  (0, 1375)	0.06406451416208277
  (0, 5165)	0.04948662696586209
  (0, 10053)	0.07866277237929355
  (0, 12241)	0.09378036099028715
  (0, 4593)	0.08675108258906822
  (0, 7523)	0.0993598131443519
  (0, 1824)	0.09378036099028715
  (0, 12003)	0.1987196262887038
  (0, 1618)	0.040226492331054506
  (0, 5586)	0.09378036099028715
  (0, 3835)	0.08675108258906822
  :	:
  (1004, 9427)	0.09174485565609045
  (1004, 6306)	0.061941948365046524
  (1004, 5058)	0.04570175683699196
  (1004, 13098)	0.09023199229675694
  (1004, 2458)	0.05848616749334612
  (1004, 11350)	

## Calculate the similarity

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
# menghitung nilai cosine similarity berdasarkan matrix tf idf
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  # Mengkonstruksi Cosine Similarity Matrix (no. of movies x no. of movies)
cosine_sim.shape

(1005, 1005)

In [ ]:
print(cosine_sim)

[[1.         0.         0.         ... 0.00637605 0.         0.01391129]
 [0.         1.         0.04311757 ... 0.         0.         0.        ]
 [0.         0.04311757 1.         ... 0.         0.         0.        ]
 ...
 [0.00637605 0.         0.         ... 1.         0.08406375 0.04843321]
 [0.         0.         0.         ... 0.08406375 1.         0.06777457]
 [0.01391129 0.         0.         ... 0.04843321 0.06777457 1.        ]]


## Recommend the film

In [ ]:
indices = pd.Series(df.index, index = df['judul_film']).drop_duplicates()

In [ ]:
def recommend_film(title, k, cosine_sim = cosine_sim):
    if df['judul_film'].isin([title]).any(): #if the movie exists in database
        idx = indices[title]
        # mendapatkan cosine similarity score berdasarkan title
        sim_scores = list(enumerate(cosine_sim[idx]))  
        sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
        sim_scores = sim_scores[1:k+1]                                        
        movie_indices = [i[0] for i in sim_scores] 

        return (df['judul_film'].iloc[movie_indices], sim_scores)
    else:
        print("Sorry, we don't have that movie in our database")

In [ ]:
# masukan judul film yang telah di tonton untuk menampilkan rekomendasi, judul film dapat diubah sesuai dengan 
# judul yang tersedia di dataframe, dan seberapa banyak film yang ingin direkomendasikan
recommend_film('sang pencerah',10)

(926          cinta di saku celana
 941          something in the way
 155            para pencari tuhan
 232               danur 2: maddah
 113                   message man
 764                catatan si boy
 782                           #66
 152                       kartini
 383    wanita harimau (santet ii)
 944     99 cahaya di langit eropa
 Name: judul_film, dtype: object,
 [(926, 0.1541814853894271),
  (941, 0.14583902158314332),
  (155, 0.14173684073483198),
  (232, 0.08905417452892607),
  (113, 0.08576576481722281),
  (764, 0.08518593184955618),
  (782, 0.08125883561477669),
  (152, 0.0803828304722528),
  (383, 0.07840231074308311),
  (944, 0.07567502473340576)])

In [ ]:
# masukan judul film yang telah di tonton untuk menampilkan rekomendasi, judul film dapat diubah sesuai dengan 
# judul yang tersedia di dataframe, dan seberapa banyak film yang ingin direkomendasikan
recommend_film('ada apa dengan cinta?',10)

(94                    lewat tengah malam
 473                   marmut merah jambu
 529                        stip & pensil
 842                          tentang dia
 953                        cinta setaman
 215    aku ingin mencintaimu sekali saja
 321          rintihan kuntilanak perawan
 154                           lima elang
 783                   bangkit dari kubur
 212                after school horror 2
 Name: judul_film, dtype: object,
 [(94, 0.15494760174344813),
  (473, 0.11748342557383275),
  (529, 0.10920258986377207),
  (842, 0.10603618081928938),
  (953, 0.10048671816375998),
  (215, 0.09959650004058776),
  (321, 0.09835349511008429),
  (154, 0.09786349956183145),
  (783, 0.09629739855641944),
  (212, 0.09358997245234887)])